In [7]:
#Importar librerías
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [8]:
#Cargar Archivo
df1 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Cordoba1")
df2 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Guajira2")
df3 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Guajira3")
df4 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Guajira4")
df5 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Antioquia5")
df6 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Antioquia6")
df7 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Atlantico7")
df8 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Atlantico8")
df9 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Magdalena9")
df10 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Magdalena10")
df11 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Cesar11")
df12 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Cesar12")
df13 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Bolivar13")
df14 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Bolivar14")
df15 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Choco15")

# Lista Df
dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15]

# Unirlos en uno solo
df = pd.concat(dfs, ignore_index=True)

In [9]:
# Dividir en X e y
X = df.drop("Toneladas por hectaria", axis=1)
y = df["Toneladas por hectaria"]

# Train-test split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Normalizacion
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(train_X.to_numpy())  

Búsqueda de Hiperparámetros

In [11]:
# Definir listas de hiperparámetros a iterar
activations = ['relu', 'tanh', 'elu', 'selu']
optimizers = ['adam', 'sgd', 'rmsprop']
learning_rates = [0.0001, 0.001, 0.005, 0.01]
losses = ['mean_absolute_error', 'mean_squared_error', 'huber', 'log_cosh']
num_layers_list = [1, 2, 3, 4]
units_list = [64, 128, 256, 512]

In [12]:
# Inicializar resultados
results = []

# Contar total de combinaciones
total_combinations = (len(activations) * len(optimizers) * len(learning_rates) * 
                      len(losses) * len(num_layers_list) * len(units_list))
current_combination = 1

# Iterar sobre todas las combinaciones
for activation in activations:
    for optimizer_name in optimizers:
        for learning_rate in learning_rates:
            for loss in losses:
                for num_layers in num_layers_list:
                    for units in units_list:

                        print(f"➡️  Ejecutando combinación {current_combination} de {total_combinations}")
                        current_combination += 1

                        # Definir modelo
                        model = tf.keras.Sequential()
                        model.add(norm_layer)

                        for _ in range(num_layers):
                            model.add(tf.keras.layers.Dense(units=units, activation=activation))

                        model.add(tf.keras.layers.Dense(1))

                        # Configurar optimizador
                        if optimizer_name == 'adam':
                            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
                        elif optimizer_name == 'sgd':
                            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
                        elif optimizer_name == 'rmsprop':
                            optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

                        # Compilar modelo
                        model.compile(optimizer=optimizer, loss=loss, metrics=['mae'])

                        # Entrenar modelo
                        history = model.fit(train_X, train_y, validation_split=0.2, epochs=50, verbose=0)

                        # Predecir
                        y_pred = model.predict(test_X).flatten()

                        # Controlar NaNs en predicciones
                        if np.isnan(y_pred).any():
                            print(f"⚠️  Combinación inválida por NaN: {activation}, {optimizer_name}, {learning_rate}, {loss}, {num_layers} capas, {units} unidades.")
                            continue

                        # Calcular métricas
                        mse = mean_squared_error(test_y, y_pred)
                        rmse = np.sqrt(mse)
                        mae = mean_absolute_error(test_y, y_pred)
                        r2 = r2_score(test_y, y_pred)

                        # Guardar resultados
                        results.append({
                            'activation': activation,
                            'optimizer': optimizer_name,
                            'learning_rate': learning_rate,
                            'loss': loss,
                            'num_layers': num_layers,
                            'units': units,
                            'MAE': mae,
                            'RMSE': rmse,
                            'R2': r2
                        })

➡️  Ejecutando combinación 1 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
➡️  Ejecutando combinación 2 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
➡️  Ejecutando combinación 3 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
➡️  Ejecutando combinación 4 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
➡️  Ejecutando combinación 5 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
➡️  Ejecutando combinación 6 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
➡️  Ejecutando combinación 7 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
➡️  Ejecutando combinación 8 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
➡️  Ejecutando combinación 9 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
➡️  Ejecutando combinación 10 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
➡️  Ejecutando combinación 11 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
➡️  Ejecutando combinación 12 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
➡️  Ejecutando combinación 13 de 3072
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
➡️  Ejecutando combin

Resultados

In [13]:
# Convertir a DataFrame
results_df = pd.DataFrame(results)

# Mostrar el mejor modelo
best_model = results_df.sort_values('MAE').iloc[0]
print("\nMejores hiperparámetros encontrados:")
print(best_model)


Mejores hiperparámetros encontrados:
activation                      elu
optimizer                       sgd
learning_rate                 0.005
loss             mean_squared_error
num_layers                        1
units                           512
MAE                        0.159198
RMSE                       0.221706
R2                         0.999963
Name: 1912, dtype: object
